# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Loan CSV file generated from WeatherPy Folder
weather_data_to_load = "../WeatherPy/weather_df.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head(20)


,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,rikitea,PF,2022-05-04 19:40:53,-23.1203,-134.9692,74.0,83.0,77.630,8.77
1,albany,US,2022-05-04 19:40:54,42.6001,-73.9662,100.0,92.0,53.654,1.63
2,brejning,DK,2022-05-04 19:40:54,55.6613,9.6770,100.0,69.0,55.688,4.51
3,ushuaia,AR,2022-05-04 19:40:54,-54.8000,-68.3000,40.0,65.0,44.492,6.17
4,cape town,ZA,2022-05-04 19:40:55,-33.9258,18.4232,0.0,87.0,64.382,4.12
5,takoradi,GH,2022-05-04 19:40:55,4.8845,-1.7554,75.0,76.0,82.922,4.51
6,kavieng,PG,2022-05-04 19:40:56,-2.5744,150.7967,75.0,77.0,82.760,4.88
7,iqaluit,CA,2022-05-04 19:40:56,63.7506,-68.5145,20.0,73.0,12.200,9.77
8,san carlos de bariloche,AR,2022-05-04 19:40:57,-41.1456,-71.3082,75.0,51.0,60.818,10.80
9,lagos,NG,2022-05-04 19:40:57,6.5833,3.7500,79.0,91.0,77.594,1.33


In [14]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_weather_data[["Latitude", "Longitude"]]

humidity = dropna_weather_data["Humidity (%)"].astype(float)

In [16]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Filter vacation with zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,santiago del estero,AR,2022-05-04 19:38:46,-27.7951,-64.2615,0.0,30.0,73.940,4.48
1,sarakhs,IR,2022-05-04 19:43:30,36.5449,61.1577,0.0,53.0,75.614,3.09
2,ladario,BR,2022-05-04 19:44:18,-19.0047,-57.6017,0.0,40.0,73.958,4.63
3,praia,CV,2022-05-04 19:44:47,14.9215,-23.5087,0.0,53.0,79.610,9.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel
0,santiago del estero,AR,2022-05-04 19:38:46,-27.7951,-64.2615,0.0,30.0,73.940,4.48,Apartamentos Castelli 90
1,sarakhs,IR,2022-05-04 19:43:30,36.5449,61.1577,0.0,53.0,75.614,3.09,مهمان سرای شرکت گاز
2,ladario,BR,2022-05-04 19:44:18,-19.0047,-57.6017,0.0,40.0,73.958,4.63,Hotel Fazenda 4 cantos
3,praia,CV,2022-05-04 19:44:47,14.9215,-23.5087,0.0,53.0,79.610,9.26,LT Aparthotel


In [20]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

Figure(layout=FigureLayout(height='420px'))